Vamos agora extrair os dados de um [link](https://gist.githubusercontent.com/guilhermesilveira/2d2efa37d66b6c84a722ea627a897ced/raw/10968b997d885cbded1c92938c7a9912ba41c615/tracking.csv), printa-lo e depois vamos partir para o treinamento dele.

A base de dados é composta por páginas de um site e no fim por uma variável que indica se interação do usuário
terminou em compra de produtos no site.


In [19]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/2d2efa37d66b6c84a722ea627a897ced/raw/10968b997d885cbded1c92938c7a9912ba41c615/tracking.csv"
dados = pd.read_csv(uri)
dados.head()

,home,how_it_works,contact,bought
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0


In [20]:
x = dados[["home","how_it_works","contact"]]
y = dados["bought"]

Feito a leitura dos dados, agora vamos fazer a divisão do dados para que possamos fazer o teste. Pra isso, vamos
treinar o sistema com 75% do dadose treinar a porcentagem restante.

In [21]:
treino_x = x[:75]
treino_y = y[:75]
teste_x = x[75:]
teste_y = y[75:]

Vamos treinar o nosso algorítmo

In [22]:
from sklearn.svm import LinearSVC

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)

LinearSVC()

Agora vamos fazer as previsões do dados restantes

In [23]:
from sklearn.metrics import accuracy_score

previsoes = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes)

print("A acurácia foi de %.2f%% " % (acuracia * 100))

A acurácia foi de 95.83% 


# Estratificando o split

In [24]:
from sklearn.model_selection import train_test_split


for i in range(6):
    treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size =0.25)
    previsoes = modelo.predict(teste_x)
    acuracia = accuracy_score(teste_y, previsoes)

    print("A acurácia foi de %.2f%% " % (acuracia * 100))

A acurácia foi de 96.00% 
A acurácia foi de 100.00% 
A acurácia foi de 100.00% 
A acurácia foi de 96.00% 
A acurácia foi de 96.00% 
A acurácia foi de 92.00% 


Ao executarmos várias vezes o algoritmo, podemos perceber que obtivemos resultado diferentes. Isso acontece porque a
cada iteração, o array gerado nos treinos e teste são aleatório.

Pra evitarmos isso e para que possamos reproduzir o mesmo resultado após repentinas chamada a mesma função, temos que
 definir um número no qual denominamos *SEED* pra que a divisão seja sempre a mesma em todas as chamadas

In [25]:
SEED = 20


for i in range(5):
    treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state = SEED, test_size = 0.25)

    modelo = LinearSVC()
    modelo.fit(treino_x, treino_y)
    previsoes = modelo.predict(teste_x)

    acuracia = accuracy_score(teste_y, previsoes) * 100
    print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 96.00%
A acurácia foi 96.00%
A acurácia foi 96.00%
A acurácia foi 96.00%
A acurácia foi 96.00%


Porém, se analisarmos as proporções entre as pessoas que compraram  e não entre os dados de treino e teste, podemos
perceber uma diferença.

In [26]:
import numpy as np
import pandas as pd

proporcao = np.zeros((2,4), dtype = object)


treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state = SEED, test_size = 0.25)

proporcao[0][0] = 'treino_y'
proporcao[0][1] = treino_y.value_counts()[0]
proporcao[0][2] = treino_y.value_counts()[1]
proporcao[0][3] ='%.2f' %  (treino_y.value_counts()[0]/treino_y.value_counts()[1])

proporcao[1][0] = 'teste_y'
proporcao[1][1] = teste_y.value_counts()[0]
proporcao[1][2] = teste_y.value_counts()[1]
proporcao[1][3] = '%.2f' % (teste_y.value_counts()[0]/teste_y.value_counts()[1])


pd.DataFrame(proporcao, columns=['',0,1, "proporcao 0/1"])

,,0,1,proporcao 0/1
0,treino_y,47,27,1.74
1,teste_y,19,6,3.17


Temos que evitaressa diferença de proporção entre as pessoas que compraram e as que não compraram. Pois, poderíamos
em algum caso, testa somente casos que não houveram comprarm (ou virce-versa)

Pra isso, adicionamos valor ao parametro **stratify** pra que ele nos ajude a diminuir essa disparidad. No nosso
caso, queremos a proporção no **eixo Y**

In [27]:
proporcao = np.zeros((2,4), dtype = object)


treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state = SEED, test_size = 0.25, stratify=y)

proporcao[0][0] = 'treino_y'
proporcao[0][1] = treino_y.value_counts()[0]
proporcao[0][2] = treino_y.value_counts()[1]
proporcao[0][3] ='%.2f' %  (treino_y.value_counts()[0]/treino_y.value_counts()[1])

proporcao[1][0] = 'teste_y'
proporcao[1][1] = teste_y.value_counts()[0]
proporcao[1][2] = teste_y.value_counts()[1]
proporcao[1][3] = '%.2f' % (teste_y.value_counts()[0]/teste_y.value_counts()[1])


pd.DataFrame(proporcao, columns=['',0,1, "proporcao 0/1"])


,,0,1,proporcao 0/1
0,treino_y,49,25,1.96
1,teste_y,17,8,2.12
